# Group Comparison analysis pRF propertis 

In [5]:
# Load the necessary libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 
import shutil
from matplotlib.backends.backend_pdf import PdfPages

In [6]:
# Define whether you want to run the code on one subject or on the whole dataset

# 1. Run the code on one specific subject 
#subjects = ['sub-22']

# 2. Run the code on glaucoma patients 
#subjects = [f'sub-{i:02}'for i in range(2, 21)]

# 3. Run the code on healthy controls 
subjects = [f'sub-{i:02}'for i in range(2, 47)]

In [7]:
# Define the main variables 
target = 'V3'
source = 'V1'
hemis = ['lh', 'rh']
atlas = 'manual'
delineation = 'manualdelin'
denoising = 'nordic_sm4'
task = 'RET'
MAIN_PATH = '/Volumes/FedericaCardillo/pre-processing/projects/EGRET+/derivatives/CFM'
poag = [f'sub-{i:02}' for i in range(2, 21)]
hc = [f'sub-{i:02}' for i in range(21, 47)]
# subjects = poag + hc
visual_areas = ['V2', 'V3', 'V4', 'LO']
freesurfer = f"{MAIN_PATH}/freesurfer"
cumulative = os.path.join(MAIN_PATH,'culumativeplots')
sigmas = []
if os.path.exists(cumulative): 
    shutil.rmtree(cumulative)
os.makedirs(cumulative)

tasks = {
    "RET": {
        "label": "RET",
        "dir": "RET",
        "color": {"HC": "blue", "POAG": "red"}
    },
    "RET2": {
        "label": "RET2",
        "dir": "RET2",
        "color": {"HC": "green", "POAG": "salmon"}
    },
    "REST": {
        "label": "REST",
        "dir": "RestingState",  # ← Correct folder name
        "color": {"HC": "black", "POAG": "gray"}
    }
}
groups = {"HC": hc, "POAG": poag}

In [8]:
def sigma(subj, visual_area, task_name):
    sigma_vals = []
    task_dir = tasks[task_name]["dir"]
    for hemi in hemis:
        path = f'{MAIN_PATH}/{subj}/ses-1/GM/{hemi}/{denoising}/{task_dir}/{visual_area}/best_fits_prf.csv'
        if not os.path.exists(path):
            continue
        df = pd.read_csv(path)
        df["Best Sigma Finer"] = pd.to_numeric(df["Best Sigma Finer"], errors='coerce')
        df["Best Variance Explained Finer"] = pd.to_numeric(df["Best Variance Explained Finer"], errors='coerce')
        df = df.dropna(subset=["Best Sigma Finer", "Best Variance Explained Finer"])
        df = df[(df["Best Sigma Finer"] > 0.05) & (df["Best Variance Explained Finer"] > 0.3)]
        sigma_vals.extend(df["Best Sigma Finer"].values)
    return sigma_vals


def individual_cdfs(group, color, label, visual_area, task_name):
    cdfs = []
    included_subjects = 0
    for subj in group:
        sigma_vals = sigma(subj, visual_area, task_name)
        #if not sigma_vals:
        #    continue
        included_subjects += 1
        sorted_sigma = np.sort(sigma_vals)
        cdf = np.linspace(0, 1, len(sorted_sigma))
        plt.plot(sorted_sigma, cdf, linestyle='--', color=color, alpha=0.4)
        grid_cdf = np.interp(x, sorted_sigma, cdf, left=0, right=1)
        cdfs.append(grid_cdf)
    if cdfs:
        median = np.median(cdfs, axis=0)
        plt.plot(x, median, color=color, linewidth=2.5, label=f"{label} Median (N={included_subjects})")
    return cdfs

In [9]:
# PLOT THE CUMULATIVE DISTRIBUTION FUNCTION REPRESENTING THE CONNECTIVE FIELD SIZE SUBJECT PER SUBJECTS
pdf_hemis = PdfPages(os.path.join(cumulative, 'hemis.pdf'))
pdf_hemi = PdfPages(os.path.join(cumulative, 'hemi.pdf'))

for subj in subjects: 
    sigma_vals = {}

    for hemi in hemis: 
        best_fit_path = f'{MAIN_PATH}/{subj}/ses-1/GM/{hemi}/{denoising}/{task}/{target}/best_fits_prf.csv'
        if not os.path.exists(best_fit_path):
            print(f'Best fit file not found for {subj} ({hemi}).')
            continue

        best_fit_df = pd.read_csv(best_fit_path)
        best_fit_df = best_fit_df[best_fit_df["Best Sigma Finer"] > 0.05]
        best_fit_df = best_fit_df[best_fit_df["Best Variance Explained Finer"] > 0.3]
        sigma_vals[hemi] = best_fit_df["Best Sigma Finer"].values

    # Plot the hemispheres separately 
    fig_one, axs = plt.subplots(1, 2, figsize=(14, 6))
    for i in range(len(hemis)):
        hemi = hemis[i]
        if hemi in sigma_vals:
            axs[i].hist(sigma_vals[hemi], bins=50, cumulative=True, density=True, alpha=0.7)
            axs[i].set_title(f"{subj} - {hemi} - {target}-{source}")
            plt.xlim(0, max(max(sigma_vals[hemi]), 1))
            axs[i].set_xlabel("Connective Field Sigma (degrees)")
            axs[i].set_ylabel("Cumulative Probability")
            axs[i].grid(True)

    fig_one.tight_layout()
    pdf_hemis.savefig(fig_one)
    plt.close(fig_one)

    # Plot the hemispheres together  
    if all(h in sigma_vals for h in hemis):
        sigma_total = list(sigma_vals['lh']) + list(sigma_vals['rh']) 
        fig_two = plt.figure(figsize=(7, 6))
        plt.hist(sigma_total, bins=50, cumulative=True, density=True, alpha=0.7, color='blue')
        plt.title(f"{subj} - {target}-{source}")
        plt.xlim(0, max(sigma_total))
        plt.xlabel("Connective Field Sigma (degrees)")
        plt.ylabel("Cumulative Probability")
        plt.grid(True)
        pdf_hemi.savefig(fig_two) 
        plt.close(fig_two)

pdf_hemis.close()
pdf_hemi.close()

Best fit file not found for sub-03 (lh).
Best fit file not found for sub-03 (rh).
Best fit file not found for sub-06 (lh).
Best fit file not found for sub-06 (rh).
Best fit file not found for sub-24 (lh).
Best fit file not found for sub-24 (rh).


In [10]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Folder to save individual plots
output_dir = os.path.join(cumulative, f"groupcomparison_{denoising}_jpg")
os.makedirs(output_dir, exist_ok=True)

# Only include specific (group, task) combinations
included_combinations = {
    ("POAG", "RET"),
    ("HC", "RET"),
    ("HC", "RET2"),
    #("HC", "REST"),
    #("POAG", "REST")
}

for visual_area in visual_areas:
    x = np.linspace(0, 10, 500)

    # A. Individual group/task combinations
    for group_name, group_subjects in groups.items():
        for task_name, task_info in tasks.items():
            if (group_name, task_name) not in included_combinations:
                continue
            plt.figure(figsize=(6, 6))
            cdfs = []
            included_subjects = 0

            for subj in group_subjects:
                sigma_vals = sigma(subj, visual_area, task_name)
                if not sigma_vals:
                    continue
                included_subjects += 1
                sorted_sigma = np.sort(sigma_vals)
                cdf = np.linspace(0, 1, len(sorted_sigma))
                plt.plot(sorted_sigma, cdf, linestyle='--', color=task_info["color"][group_name], alpha=0.4)
                grid_cdf = np.interp(x, sorted_sigma, cdf, left=0, right=1)
                cdfs.append(grid_cdf)

            if cdfs:
                median = np.median(cdfs, axis=0)
                label = f"{group_name} {task_info['label']} Median (N={included_subjects})"
                plt.plot(x, median, color=task_info["color"][group_name], linewidth=2.5, label=label)

            plt.title(f"{visual_area} - {group_name} - {task_name}")
            plt.xlabel("Connective Field Sigma (mm)")
            plt.ylabel("Cumulative Probability")
            plt.grid(True)
            plt.legend()
            plot_path = os.path.join(output_dir, f"{visual_area}_{group_name}_{task_name}.jpg")
            plt.savefig(plot_path, dpi=300)
            plt.close()

    # B. All groups/tasks in one plot
    plt.figure(figsize=(7, 7))
    plotted_anything = False

    for group_name, group_subjects in groups.items():
        for task_name, task_info in tasks.items():
            if (group_name, task_name) not in included_combinations:
                continue
            cdfs = []
            included_subjects = 0

            for subj in group_subjects:
                sigma_vals = sigma(subj, visual_area, task_name)
                if not sigma_vals:
                    continue
                included_subjects += 1
                sorted_sigma = np.sort(sigma_vals)
                cdf = np.linspace(0, 1, len(sorted_sigma))
                grid_cdf = np.interp(x, sorted_sigma, cdf, left=0, right=1)
                cdfs.append(grid_cdf)

            if cdfs:
                median = np.median(cdfs, axis=0)
                label = f"{group_name} {task_info['label']} (N={included_subjects})"
                plt.plot(x, median, color=task_info["color"][group_name], linewidth=2.5, label=label)
                plotted_anything = True

    plt.title(f"{visual_area} - All Groups & Tasks Comparison {denoising}")
    plt.xlabel("Connective Field Sigma (mm)")
    plt.ylabel("Cumulative Probability")
    plt.grid(True)
    if plotted_anything:
        plt.legend()
    else:
        plt.text(0.5, 0.5, "No data available", ha='center', va='center', transform=plt.gca().transAxes)

    summary_plot_path = os.path.join(output_dir, f"{visual_area}_all_comparison.jpg")
    plt.savefig(summary_plot_path, dpi=300)
    plt.close()

In [11]:
# Now create all-groups/tasks comparison for eccentricity
for visual_area in visual_areas:
    plt.figure(figsize=(7, 7))
    plotted_anything = False

    for group_name, group_subjects in groups.items():
        for task_name, task_info in tasks.items():
            if (group_name, task_name) not in included_combinations:
                continue
            cdfs = []
            included_subjects = 0

            for subj in group_subjects:
                ecc_vals = []
                task_dir = task_info["dir"]
                for hemi in hemis:
                    path = f'{MAIN_PATH}/{subj}/ses-1/GM/{hemi}/{denoising}/{task_dir}/{visual_area}/best_fits_prf.csv'
                    if not os.path.exists(path):
                        continue
                    df = pd.read_csv(path)
                    df["Source Eccentricity"] = pd.to_numeric(df["Source Eccentricity"], errors='coerce')
                    df["Best Variance Explained Finer"] = pd.to_numeric(df["Best Variance Explained Finer"], errors='coerce')
                    df = df.dropna(subset=["Source Eccentricity", "Best Variance Explained Finer"])
                    df = df[df["Best Variance Explained Finer"] > 0.3]
                    ecc_vals.extend(df["Source Eccentricity"].values)

                if ecc_vals:
                    included_subjects += 1
                    sorted_ecc = np.sort(ecc_vals)
                    cdf = np.linspace(0, 1, len(sorted_ecc))
                    grid_cdf = np.interp(x_ecc, sorted_ecc, cdf, left=0, right=1)
                    cdfs.append(grid_cdf)

            if cdfs:
                median = np.median(cdfs, axis=0)
                label = f"{group_name} {task_info['label']} (N={included_subjects})"
                plt.plot(x_ecc, median, color=task_info["color"][group_name], linewidth=2.5, label=label)
                plotted_anything = True

    plt.title(f"{visual_area} - All Groups & Tasks Comparison (Eccentricity)")
    plt.xlabel("Eccentricity (deg)")
    plt.ylabel("Cumulative Probability")
    plt.grid(True)
    if plotted_anything:
        plt.legend()
    else:
        plt.text(0.5, 0.5, "No data available", ha='center', va='center', transform=plt.gca().transAxes)

    summary_plot_path = os.path.join(ecc_output_dir, f"{visual_area}_all_comparison_ecc.jpg")
    plt.savefig(summary_plot_path, dpi=300)
    plt.close()

NameError: name 'x_ecc' is not defined

<Figure size 700x700 with 0 Axes>